# SANTANDER VALUE PREDICTION

#### IMPORTING THE NECESSARY PACKAGES

In [1]:
# GENERAL LIBRARIES
import pandas as pd 
import csv 
from pandas import DataFrame as df
import numpy as np

# FEATURE ENGINEERING
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# MODEL DEPLOYMENT
import keras
from keras.models import Sequential 
from keras.layers import Dense , Dropout
from sklearn.cross_validation import train_test_split 
#from sklearn.linear_model import LinearRegression

# MODEL TUNING
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.
C:\Users\Dell-pc\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### IMPORTING DATASET

In [3]:
# train dataset

data_1 = pd.read_csv('train.csv')
output_target = data_1.target # value to be predicted in the test dataset
data_1 = data_1.drop('target',axis=1)

In [9]:
data_1.head(5)

,0,1,2,3,4,5,6,7,8,9,...,4982,4983,4984,4985,4986,4987,4988,4989,4990,4991
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006883,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006258,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
data_1.shape

(4459, 4992)

In [19]:
# test dataset 
data_2 = pd.read_csv('test.csv')
data_2.shape

(49342, 4992)

In [ ]:
data_2.head(5)

### FEATURE ENGINEERING 

In [7]:
def train(train_data):
    # label encoding
    le = LabelEncoder()
    ID = le.fit_transform(train_data.ID)
    data_1 = train_data.drop('ID' , axis = 1)
    ID = pd.DataFrame(ID, columns=['ID'])
    data_1 = ID.join(data_1)
    
    # normalising dataset
    scale = MinMaxScaler()
    data_1 = scale.fit_transform(data_1)
    data_1 = pd.DataFrame(data_1)
    
    return data_1 

data_1 = train(data_1)

In [8]:
data_1.head(3)

,0,1,2,3,4,5,6,7,8,9,...,4982,4983,4984,4985,4986,4987,4988,4989,4990,4991
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006883,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
def target(target_dataset):
    output_target = target_dataset.reshape(-1,1)
    scale = MinMaxScaler()
    output_target = scale.fit_transform(output_target)
    output_target = pd.DataFrame(output_target)
    
    return output_target

output_target = target(output_target)

C:\Users\Dell-pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [22]:
def test(test_dataset):
    # label encoding
    le = LabelEncoder() 
    ID = le.fit_transform(test_dataset.ID)
    data_2 = data_2.drop('ID',axis=1)
    ID = pd.DataFrame(ID,columns=['ID'])
    data_2 = ID.join(data_2)
    
    # normalising dataset
    scale = MinMaxScaler()
    data_2 = scale.fit_transform(data_2)
    data_2 = pd.DataFrame(data_2)
    
    return data_2

data_2 = test(data_2)

### MODEL 

In [25]:
x_train , x_test , y_train ,y_test = train_test_split(data_1 , output_target , test_size=0.3,random_state=0)

def mlp_regressor():
    regressor = Sequential()
    regressor.add(Dense(output_dim=1000,init='uniform',activation='relu',input_dim=4992))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(output_dim=500,init='uniform',activation='relu',input_dim=1000))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(output_dim=100,init='uniform',activation='relu',input_dim=500))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(output_dim=10,init='uniform',activation='relu',input_dim=100))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(output_dim=1,init='uniform',activation='linear'))
    regressor.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])
    
    #regressor.summary()
    return regressor

regressor.fit(x_train , y_train ,epochs=200,batch_size=64)
      

### PREDICTION 

In [ ]:
def predicted_target(data_test):
    values = regressor.predict(data_test)
    values = abs(values)
    
    return values

values = predicted_target(data_2)

#### Formatting the dataset

In [ ]:
def final_dataset(data):
    final_df = le.inverse_transform(data.ID)
    final_df = pd.DataFrame(final_df , columns=['ID'])
    final_df_2 = pd.DataFrame(values , columns=['Target'])
    final_test_target = final_df.join(final_df_2)
    
    return final_test_target

dataset = final_dataset(data_2)
final_test_target.to_csv('final_target.csv' , index=False)

### Calculating the score 

In [ ]:
parameters = {'batch_size' :[20,32],
              'nb_epoch' : [50,150],
              'optimizer':['adam','rmsprop']
              }
regressor_evaluating = KerasRegressor(mlp_regressor)
grid_search = GridSearchCV(estimator = regressor_evaluating,
                           param_grid = parameters , 
                           scoring = 'neg_mean_squared_error',
                           cv=10)
grid_search = grid_search.fit(x_train , y_train)
grid_search_parameter = grid_search.best_params_
grid_search_score = grid_search.best_score_

### loss : 0.0045 ; mean_absolute_error : 0.043